# Crawling PTT
爬取 PTT JOB 版的文章內容

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.ptt.cc/bbs/job/index.html"
response = requests.get(url, timeout=(2, 3))
response.status_code

200

In [2]:
# print(response.text)

## Converting webpage to a soup object

In [3]:
soup = BeautifulSoup(response.text, "html.parser")
print(type(soup))
# print(soup.prettify())

<class 'bs4.BeautifulSoup'>


In [4]:
print(soup.title)
print(soup.title.name)
print(soup.title.string)
print(soup.title.parent.name)

<title>看板 job 文章列表 - 批踢踢實業坊</title>
title
看板 job 文章列表 - 批踢踢實業坊
head


## Find all links

In [5]:
print(type(soup.find_all("a")))
print(len(soup.find_all("a")))
soup.find_all("a")[:5]

<class 'bs4.element.ResultSet'>
67


[<a href="/bbs/" id="logo">批踢踢實業坊</a>,
 <a class="board" href="/bbs/job/index.html"><span class="board-label">看板 </span>job</a>,
 <a class="right small" href="/about.html">關於我們</a>,
 <a class="right small" href="/contact.html">聯絡資訊</a>,
 <a class="btn selected" href="/bbs/job/index.html">看板</a>]

In [6]:
nodes_a = soup.find_all("a")
print(nodes_a[0])
print(nodes_a[0].text)
print(nodes_a[0].get('href'))

<a href="/bbs/" id="logo">批踢踢實業坊</a>
批踢踢實業坊
/bbs/


In [7]:
links = []
for link in nodes_a:
    links.append(link.get('href'))
print(len(links))

67


## Find titles of posts
* `.find_all(class_="...")`
此處觀察html檔，文章在`class_="r-ent"`中，`r-ent`下
    * `class="nrec"` 推文數
    * `class="title"` 文章標題
    * `class="author"` 作者
    * `class="date"` 日期


In [8]:
import requests
from bs4 import BeautifulSoup

url = "https://www.ptt.cc/bbs/job/index.html"
response = requests.get(url, timeout=(2, 3))
soup = BeautifulSoup(response.text, "html.parser")

In [9]:
print(len(soup.find_all(class_="r-ent")))

19


In [10]:
for div in soup.find_all(class_="r-ent"):
    print(type(div))
    print("="*30)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [11]:
for div in soup.find_all(class_ = "r-ent")[:5]:
    print(div.find(class_="nrec").text.strip())
    print(div.find(class_="title").text.strip())
    print(div.find(class_="author").text.strip())
    print(div.find(class_="date").text.strip())
    print(div.find(class_="title").a.get("href"))
    print("="*30)

1
[台北] 源流科技/衍生性金融商品交易員
carol271216
6/23
/bbs/job/M.1655976587.A.B2D.html

[台北] 公館 門市人員
asd33321
6/23
/bbs/job/M.1655976617.A.83B.html

[台北] 維他露誠徵量販賣場陳列員（信亞人力）
betty1127
6/23
/bbs/job/M.1655981612.A.0E5.html

[台北] 台灣拓人個別指導教育 儲備幹部
koftodd
6/23
/bbs/job/M.1655983393.A.6E5.html

[台南] 關廟區-超商理貨人員
hiimck
6/23
/bbs/job/M.1655985872.A.AEA.html


### If the link has been removed

In [37]:
prefix  = "http://www.ptt.cc/"
for div in soup.find_all(class_ = "r-ent")[:5]:
    print(div.find(class_="nrec").text.strip())
    print(div.find(class_="title").text.strip())
    print(div.find(class_="author").text.strip())
    print(div.find(class_="date").text.strip())
    #　print(div.find(class_="title").a.get("href"))
    
    try:
        print(prefix + div.find(class_="title").a.get("href"))
    except:
        print("The link has been removed")
    print("="*30)

1
[台北] 源流科技/衍生性金融商品交易員
carol271216
6/23
http://www.ptt.cc//bbs/job/M.1655976587.A.B2D.html

[台北] 公館 門市人員
asd33321
6/23
http://www.ptt.cc//bbs/job/M.1655976617.A.83B.html

[台北] 維他露誠徵量販賣場陳列員（信亞人力）
betty1127
6/23
http://www.ptt.cc//bbs/job/M.1655981612.A.0E5.html

[台北] 台灣拓人個別指導教育 儲備幹部
koftodd
6/23
http://www.ptt.cc//bbs/job/M.1655983393.A.6E5.html

[台南] 關廟區-超商理貨人員
hiimck
6/23
http://www.ptt.cc//bbs/job/M.1655985872.A.AEA.html


## 儲存文章資訊

In [12]:
prefix = "https://www.ptt.cc/"
url = url = "https://www.ptt.cc/bbs/job/index.html"
response = requests.get(url, timeout=(2, 3))
response.status_code
soup = BeautifulSoup(response.text, "html.parser")

In [13]:
links = []
for div in soup.find_all(class_="r-ent"):
    # print(div.find(class_="nrec").text.strip())
    # print(div.find(class_="title").text.strip())
    # print(div.find(class_="author").text.strip())
    # print(div.find(class_="date").text.strip())
    
    nrec = div.find(class_="nrec").text.strip()
    title = div.find(class_="title").text.strip()
    author = div.find(class_="author").text.strip()
    date = div.find(class_="date").text.strip()
    
    try: 
        link = prefix + div.find(class_="title").a.get("href")
        temp = {"nrec": nrec,
            "title": title,
            "author": author,
            "date": date,
            "link": link
            }
        links.append(temp)
    except:
        print(div.find(class_="title").text.strip())  # 本文已被刪除

print(links[0])

{'nrec': '1', 'title': '[台北] 源流科技/衍生性金融商品交易員', 'author': 'carol271216', 'date': '6/23', 'link': 'https://www.ptt.cc//bbs/job/M.1655976587.A.B2D.html'}


In [15]:
import pandas as pd
pd.DataFrame(links).head()

,nrec,title,author,date,link
0,1,[台北] 源流科技/衍生性金融商品交易員,carol271216,6/23,https://www.ptt.cc//bbs/job/M.1655976587.A.B2D...
1,,[台北] 公館 門市人員,asd33321,6/23,https://www.ptt.cc//bbs/job/M.1655976617.A.83B...
2,,[台北] 維他露誠徵量販賣場陳列員（信亞人力）,betty1127,6/23,https://www.ptt.cc//bbs/job/M.1655981612.A.0E5...
3,,[台北] 台灣拓人個別指導教育 儲備幹部,koftodd,6/23,https://www.ptt.cc//bbs/job/M.1655983393.A.6E5...
4,,[台南] 關廟區-超商理貨人員,hiimck,6/23,https://www.ptt.cc//bbs/job/M.1655985872.A.AEA...


# Get more links from more pages
觀察出url中index後的數字為最後的頁碼

ex. https://www.ptt.cc/bbs/job/index.html, https://www.ptt.cc/bbs/job/index537.html

## Get the lastpage id

In [16]:
import requests
from bs4 import BeautifulSoup
url = "https://www.ptt.cc/bbs/job/index.html"
response = requests.get(url, timeout = (2, 3))
soup = BeautifulSoup(response.text, "html.parser")

In [17]:
# find last page number
last2nd_url = soup.find_all(class_ = "btn")[3]  # 上頁
last2nd_url = last2nd_url.get('href')  # /bbs/job/index537.html

import re
lastpage = int(re.search('index(.+)\.html', last2nd_url).group(1))+1  # (.+) '.'任意字; '+'至少一個字
lastpage

538

In [18]:
prefix = "https://www.ptt.cc"

posts = []

for i in range(lastpage, lastpage-10, -1):
    url = "https://www.ptt.cc/bbs/job/index{}.html".format(i)
    
    response = requests.get(url, timeout=(2, 3))
    soup = BeautifulSoup(response.text, "html.parser")
    
    for div in soup.find_all(class_="r-ent"):
        nrec = div.find(class_="nrec").text.strip()
        title = div.find(class_="title").text.strip()
        author = div.find(class_="author").text.strip()
        date = div.find(class_="date").text.strip()

        try: 
            link = prefix + div.find(class_="title").a.get("href")
            # print(link)
            temp = {"nrec": nrec,
                "title": title,
                "author": author,
                "date": date,
                "link": link
                }
            posts.append(temp)
        except:
            print(div.find(class_="title").text.strip())
            pass
    print(len(posts))

19
(本文已被刪除) [sely]
38
jameswade: 違反人資條款 水桶
OliverChang: 違反人資條款 水桶
OliverChang: 違反人資條款 水桶
OliverChang: 違反人資條款 水桶
OliverChang: 違反人資條款 水桶
MadlyOrange: 不可拒收站內信
(本文已被刪除) [carol271216]
isaac5566: 月休過低 請補加班費
50
(本文已被刪除) [wangteacher]
69
shipl79: 缺標語
shan520: 缺標語
advancejc: 月休過低 請補加班費計算
leobigpig: 缺標語
85
bonnielin526: 時薪工作 水桶
sng1082622: 時薪工作 水桶
js255076: 缺標語
YenHs: 缺標語
shujia: 違反人資條款
100
(本文已被刪除) [isaac5566]
(本文已被刪除) [crewfion]
118
greengo: 月休不詳 工時不詳
137
(本文已被刪除) [cin914075]
156
176


In [19]:
import pandas as pd
post_df = pd.DataFrame(posts)
post_df

,nrec,title,author,date,link
0,1,[台北] 源流科技/衍生性金融商品交易員,carol271216,6/23,https://www.ptt.cc/bbs/job/M.1655976587.A.B2D....
1,,[台北] 公館 門市人員,asd33321,6/23,https://www.ptt.cc/bbs/job/M.1655976617.A.83B....
2,,[台北] 維他露誠徵量販賣場陳列員（信亞人力）,betty1127,6/23,https://www.ptt.cc/bbs/job/M.1655981612.A.0E5....
3,,[台北] 台灣拓人個別指導教育 儲備幹部,koftodd,6/23,https://www.ptt.cc/bbs/job/M.1655983393.A.6E5....
4,,[台南] 關廟區-超商理貨人員,hiimck,6/23,https://www.ptt.cc/bbs/job/M.1655985872.A.AEA....
...,...,...,...,...,...
171,,[台北] PressPlay 數據分析師（可遠端,saftaro,6/13,https://www.ptt.cc/bbs/job/M.1655106475.A.A69....
172,3,[台北] PressPlay 產品測試專員（可遠端,saftaro,6/13,https://www.ptt.cc/bbs/job/M.1655106661.A.61C....
173,,台北 PressPlay SDET(可遠端,saftaro,6/13,https://www.ptt.cc/bbs/job/M.1655106789.A.8C8....
174,,[台北] PressPlay 產品總監（可遠端,saftaro,6/13,https://www.ptt.cc/bbs/job/M.1655107275.A.556....


# Getting post content
- `.find(id = "...")` , `.find(class = "...")` 觀察po文的html檔
    * 作者, 標題, 時間在 `class = "article-meta-value"`中
    * 文章內容在`id = "main-content"`中

In [20]:
from bs4 import NavigableString
links = list(post_df["link"])
all_posts = []

for link in links[:5]:
    print(link)
    response = requests.get(link, timeout=(2, 3))
    soup = BeautifulSoup(response.text, "html.parser")
    
    print(soup.title.text)
    
    title = soup.title.text
    metas = soup.find_all(class_ = "article-meta-value")
    # print(metas[0].text)  # 作者
    # print(metas[2].text)  # 標題
    # print(metas[3].text)  # 時間
    
    author = metas[0].text
    time = metas[3].text
    
    maintext = soup.find(id = "main-content") 
    content = ""
    # print(maintext)
    
    for text in maintext:
        if isinstance(text, NavigableString):
            content += text.strip()
    # print(content)
    
    temp = {"author": author,
            "title": title,
            "time": time,
            "link": link,
            "content": content
            }
    all_posts.append(temp)
    
    print("-"*10)
    

https://www.ptt.cc/bbs/job/M.1655976587.A.B2D.html
[台北] 源流科技/衍生性金融商品交易員 - 看板 job - 批踢踢實業坊
----------
https://www.ptt.cc/bbs/job/M.1655976617.A.83B.html
[台北] 公館 門市人員 - 看板 job - 批踢踢實業坊
----------
https://www.ptt.cc/bbs/job/M.1655981612.A.0E5.html
 [台北] 維他露誠徵量販賣場陳列員（信亞人力） - 看板 job - 批踢踢實業坊
----------
https://www.ptt.cc/bbs/job/M.1655983393.A.6E5.html
[台北] 台灣拓人個別指導教育 儲備幹部 - 看板 job - 批踢踢實業坊
----------
https://www.ptt.cc/bbs/job/M.1655985872.A.AEA.html
[台南] 關廟區-超商理貨人員 - 看板 job - 批踢踢實業坊
----------


In [21]:
import pandas as pd
df_all_posts = pd.DataFrame(all_posts)
df_all_posts

,author,title,time,link,content
0,carol271216 (卡洛卡妮),[台北] 源流科技/衍生性金融商品交易員 - 看板 job - 批踢踢實業坊,Thu Jun 23 17:29:45 2022,https://www.ptt.cc/bbs/job/M.1655976587.A.B2D....,請各位資方配合遵守。源流科技股份有限公司衍生性金融商品交易員1.負責衍生性金融商品交易作業\...
1,asd33321 (小護士),[台北] 公館 門市人員 - 看板 job - 批踢踢實業坊,Thu Jun 23 17:30:15 2022,https://www.ptt.cc/bbs/job/M.1655976617.A.83B....,請各位資方配合遵守。比樂懿股份有限公司Dr.情趣生活超商 門市人員門市產品介紹、銷售、客服\...
2,betty1127 (nn),[台北] 維他露誠徵量販賣場陳列員（信亞人力） - 看板 job - 批踢踢實業坊,Thu Jun 23 18:53:30 2022,https://www.ptt.cc/bbs/job/M.1655981612.A.0E5....,job版禁止張貼違反「就業服務法」、「性別平等工作法」、「勞基法」與其他法律之文\n章\n發...
3,koftodd (陶德),[台北] 台灣拓人個別指導教育 儲備幹部 - 看板 job - 批踢踢實業坊,Thu Jun 23 19:23:11 2022,https://www.ptt.cc/bbs/job/M.1655983393.A.6E5....,請各位資方配合遵守。台灣拓人--中和南山校/文山景興校正職儲備幹部1、協助教室室長業務。\n...
4,hiimck (hiimck),[台南] 關廟區-超商理貨人員 - 看板 job - 批踢踢實業坊,Thu Jun 23 20:04:30 2022,https://www.ptt.cc/bbs/job/M.1655985872.A.AEA....,「」、請各位資方配合遵守。巨群企業行理貨人員超商常溫商品理貨、盤點\n（泡麵、飲料、水、杯子...


## Get post comments
- `.find(id=...)`, `.find(class=...)`
    * comment 在 `class = "push-content"`

In [22]:
from bs4 import NavigableString
links = list(post_df["link"])
all_posts = []

for link in links[:10]:
    print(link)
    response = requests.get(link, timeout=(2, 3))
    soup = BeautifulSoup(response.text, "html.parser")

    metas = soup.find_all(class_ = "article-meta-value")
    author = metas[0].text
    title = metas[2].text
    time = metas[3].text
    print(title)
    
    maintext = soup.find(id = "main-content") 
    content = ""
    # print(maintext)
    
    for text in maintext:
        if isinstance(text, NavigableString):
            content += text.strip()
    # print(content)
    
    
    # 存comment
    comments_list = []
    comment = soup.find_all(class_="push-content")
    for i in range(len(comment)):
        comments_list.append(comment[i].text.strip())
    temp = {"author": author,
            "title": title,
            "time": time,
            "link": link,
            "content": content,
            "comment": comment
            }
    all_posts.append(temp)
    
    print("-"*10)
    

https://www.ptt.cc/bbs/job/M.1655976587.A.B2D.html
[台北] 源流科技/衍生性金融商品交易員
----------
https://www.ptt.cc/bbs/job/M.1655976617.A.83B.html
[台北] 公館 門市人員
----------
https://www.ptt.cc/bbs/job/M.1655981612.A.0E5.html
 [台北] 維他露誠徵量販賣場陳列員（信亞人力）
----------
https://www.ptt.cc/bbs/job/M.1655983393.A.6E5.html
[台北] 台灣拓人個別指導教育 儲備幹部
----------
https://www.ptt.cc/bbs/job/M.1655985872.A.AEA.html
[台南] 關廟區-超商理貨人員
----------
https://www.ptt.cc/bbs/job/M.1655987321.A.182.html
[南部] 台南升學補習班徵企劃
----------
https://www.ptt.cc/bbs/job/M.1655987407.A.B19.html
[中部] 彰濱工業區CAD繪圖專員
----------
https://www.ptt.cc/bbs/job/M.1655991941.A.C8C.html
[中部] 台中大雅區 理貨人員
----------
https://www.ptt.cc/bbs/job/M.1656004649.A.295.html
[台中]  zzsleeper 徵客服人員
----------
https://www.ptt.cc/bbs/job/M.1656039237.A.9DB.html
[台北] 台大醫院人體生物資料庫研究助理
----------


In [23]:
import pandas as pd
df_all_posts = pd.DataFrame(all_posts)
df_all_posts

,author,title,time,link,content,comment
0,carol271216 (卡洛卡妮),[台北] 源流科技/衍生性金融商品交易員,Thu Jun 23 17:29:45 2022,https://www.ptt.cc/bbs/job/M.1655976587.A.B2D....,請各位資方配合遵守。源流科技股份有限公司衍生性金融商品交易員1.負責衍生性金融商品交易作業\...,[[: 都是無聲卡]]
1,asd33321 (小護士),[台北] 公館 門市人員,Thu Jun 23 17:30:15 2022,https://www.ptt.cc/bbs/job/M.1655976617.A.83B....,請各位資方配合遵守。比樂懿股份有限公司Dr.情趣生活超商 門市人員門市產品介紹、銷售、客服\...,[]
2,betty1127 (nn),[台北] 維他露誠徵量販賣場陳列員（信亞人力）,Thu Jun 23 18:53:30 2022,https://www.ptt.cc/bbs/job/M.1655981612.A.0E5....,job版禁止張貼違反「就業服務法」、「性別平等工作法」、「勞基法」與其他法律之文\n章\n發...,[[: 推]]
3,koftodd (陶德),[台北] 台灣拓人個別指導教育 儲備幹部,Thu Jun 23 19:23:11 2022,https://www.ptt.cc/bbs/job/M.1655983393.A.6E5....,請各位資方配合遵守。台灣拓人--中和南山校/文山景興校正職儲備幹部1、協助教室室長業務。\n...,[]
4,hiimck (hiimck),[台南] 關廟區-超商理貨人員,Thu Jun 23 20:04:30 2022,https://www.ptt.cc/bbs/job/M.1655985872.A.AEA....,「」、請各位資方配合遵守。巨群企業行理貨人員超商常溫商品理貨、盤點\n（泡麵、飲料、水、杯子...,[]
5,vickifff (丸子不能吃),[南部] 台南升學補習班徵企劃,Thu Jun 23 20:28:39 2022,https://www.ptt.cc/bbs/job/M.1655987321.A.182....,「」、請各位資方配合遵守。\n\n\n\n【公司名稱】\n台南市市私立偉文文理短期補習班【工...,"[[: 這助理真可憐], [: 助理可憐], [: 偉哉台南]]"
6,cocawowa (書籤),[中部] 彰濱工業區CAD繪圖專員,Thu Jun 23 20:30:05 2022,https://www.ptt.cc/bbs/job/M.1655987407.A.B19....,請各位資方配合遵守。金氏科訊CAD繪圖師工程繪圖\n\n 沒有內容會被刪文具A...,[]
7,yahaha517 (拳頭大是大爺),[中部] 台中大雅區 理貨人員,Thu Jun 23 21:45:38 2022,https://www.ptt.cc/bbs/job/M.1655991941.A.C8C....,這份工作原本是團隊人員輪流處理，但現在單子慢慢多出來\n 大家決議擠出團隊經費...,[]
8,jeromeee (CXZ),[台中] zzsleeper 徵客服人員,Fri Jun 24 01:17:26 2022,https://www.ptt.cc/bbs/job/M.1656004649.A.295....,請各位資方配合遵守。zZ物理治療師客製床墊/睡羅漢有限公司線上客服1. 回覆顧客訊息及處理顧...,[]
9,yeechen (whisperer),[台北] 台大醫院人體生物資料庫研究助理,Fri Jun 24 10:53:55 2022,https://www.ptt.cc/bbs/job/M.1656039237.A.9DB....,請各位資方配合遵守。台大醫院人體生物資料庫計畫約用學/碩士級專任研究助理1. 至手術室收集病...,[]
